In [14]:
import pandas as pd
import requests
import asyncio
import json
import time
import calendar

Get latest geolocated data

Sort newest ungeolocated data

Geolocate ungeolcoated data

Output newest geolocated data to geojson

Output merged dataset to geojson

In [15]:
import pandas as pd, geopandas as gpd
import asyncio
import requests
import numpy as np

In [16]:
to_geocode = pd.read_csv("../to_geocode_datasets/1686573573-@poisk_in_ua-to_geocode.csv")

to_geocode.set_index(['id'],inplace=True)
output_dataset = to_geocode
output_dataset['coordinates'] = ""


In [17]:
def nominatim_geocode(entry, id):
    params = {
        "q" : entry["location"],
        "format" : "jsonv2"
    }
    endpoint = "https://nominatim.openstreetmap.org/search/?"
    response = requests.get(endpoint, params)
    
    if response.status_code == 200:
        print(entry)
        json_data = response.json()[0]
        coordinates = ', '.join([json_data["lat"], json_data["lon"]])
        output_dataset.at[id, "coordinates"] = coordinates
        return response.json() 
    else:
        print("error")
        print(response.status_code)
        return None
    


In [18]:
loop = asyncio.get_event_loop()
for index, entry in to_geocode.iterrows():
        res = loop.run_in_executor(None, nominatim_geocode, entry, index)

In [22]:
timestamp = calendar.timegm(time.gmtime())
output_dataset.loc[output_dataset["coordinates"] == ""].to_csv(f"../geolocated_datasets/{timestamp}-failed_geolocation.csv")
output_dataset.loc[output_dataset["coordinates"] != ""].to_csv(f"../geolocated_datasets/{timestamp}-geolocated_update.csv")

TypeError: bad operand type for unary ~: 'str'